#                        Получение данных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
df=pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')

Посмотрим на первые 10 строк данных

In [ ]:
df.head(10)

Общая информация о данных таблицы

In [ ]:
df.info()

Всего в таблице 15 столбцов, во многих из них есть пропущенные значения

#                                       Предобработка данных

Для начала удалим столбцы, которые точно не понадобятся для процесса анализа

In [ ]:
df.drop(['source_name','source_website','daily_vaccinations_raw'],axis=1,inplace=True)

Проверим, что все верно

In [ ]:
df.head()

Теперь разберемся с пропущенными значениями

In [ ]:
df.isnull().sum()

В данных достаточно много пропущенных значений, их удаление  приведет к менее точному анализу, однако из данных видно, что важные столбцы, такие как total_vaccinations и people_vaccinated складываются из столбца daily_vaccinations, у которого мало пропусков. Поэтому удалим только те строки, где значение daily_vaccinations пропущено.

In [ ]:
df=df.dropna(axis=0,subset=['daily_vaccinations'])
df.shape

# Анализ данных


# Какая страну какую вакцину использует ?

Посмотрим, какие вакцины самые популярные по странам

In [ ]:
df_vaccines=df.groupby('vaccines').agg({'country':'nunique'}).reset_index().sort_values(by='country',ascending=False)
plt.figure(figsize=(12,8))
sns.barplot(data=df_vaccines,x='vaccines',y='country').set_xticklabels(df_vaccines.vaccines,rotation=90)
plt.title('Распространенность вакцин\n',fontsize=26)
plt.xlabel('Вакцина',fontsize=18)
plt.ylabel('Число стран\n',fontsize=18)

Видно, что Pfizer/BioNTech и Oxford/AstraZeneca являются самыми популярными вакцинами

Посмотрим теперь в каких странах используются эти вакцины и построим карту мира, которая показывает их использование .

In [ ]:
df.groupby('country').agg({'vaccines':'unique'}).reset_index()


In [ ]:
import plotly.express as px
fig = px.choropleth(locations=df['country'], 
                    locationmode="country names",
                    color=df['vaccines'],
                    title="Страны по использованию вакцин",
                    height=400,
                )
fig.update_layout({'legend_orientation':'v'})
fig.update_layout({'legend_title':'Vaccine scheme'})
fig.show()

# Вывод
Вакцина Pfizer/BionTech является самой распространенной вакциной, ей пользуются в Северной Америке, Скандинавии, Европе и Австралии.
Oxford/AstraZeneca находит свое применение в Европе, Скандинавии, некоторых азиатских странах.
Вакциной Sputnik V пользуются в России и странах Южной Америки

# В каких странах программа вакцинации более продвинута?

Будем считать за "продвинутость" сумму дневных вакцинаций за период

In [ ]:
df_sum_vaccinations=df.groupby('country').agg({'daily_vaccinations':'sum'}).sort_values(by='daily_vaccinations',ascending=False).reset_index().head(10)
plt.figure(figsize=(12,8))
sns.set(font_scale=1)
sns.barplot(data=df_sum_vaccinations,x='country',y='daily_vaccinations').set_title('Число вакцин по странам\n',fontsize=26)
plt.xlabel('\nСтрана',fontsize=18)
plt.ylabel('Число вакцин\n',fontsize=18)


На графике видно число топ-10 стран по числу вакцин за период, лидером является США, в тройку также вошли Китай и Индия. Очевидно, что такое большое число вакцин связано с большим числом людей, живущих в этих странах. Поэтому продвинутость программы вакцинации лучше оценивать по числу вакцин на миллион человек.

# Где наибольшее число вакцин по отношению к населению и где привита наибольшая часть населения ?

Вспомним, как выглядят данные

In [ ]:
df.head()

Посчитаем число вакцин на 1 млн человек

In [ ]:
df_daily_vaccinations_per_million=df.groupby('country').agg({'daily_vaccinations_per_million':'sum'}).sort_values(by='daily_vaccinations_per_million',ascending=False).reset_index().head(20)
plt.figure(figsize=(12,8))
sns.set(font_scale=1)
sns.barplot(data=df_daily_vaccinations_per_million,x='country',y='daily_vaccinations_per_million').set_xticklabels(df_daily_vaccinations_per_million.country,rotation=90)
plt.title('Число вакцин на 1млн человек\n',fontsize=26)
plt.xlabel('\nСтрана',fontsize=18)
plt.ylabel('Число вакцин\n',fontsize=18)

Лидером по числу вакцин на 1 млн человек является Гибралтар, число вакцин даже превышает население, возможно страна занимается поставкой вакцин, либо предлагает привиться иностранцам. В тройку также вошли Израиль и Сейшелы.
Заметно, что в число топ-20 вошли в основном страны с небольшим населением, это и является причиной столь высоких показателей.

Теперь взглянем на долю вакцинированного населения

In [ ]:
df_total_vaccinations_per_hundred=df.groupby('country').agg({'people_vaccinated_per_hundred':'max'}).sort_values(by='people_vaccinated_per_hundred',ascending=False).reset_index().head(25)
plt.figure(figsize=(12,8))
sns.set(font_scale=1)
sns.barplot(data=df_total_vaccinations_per_hundred,x='country',y='people_vaccinated_per_hundred').set_xticklabels(df_total_vaccinations_per_hundred.country,rotation=90)
plt.title('Число вакцинированных на 100 человек\n',fontsize=26)
plt.xlabel('\nСтрана',fontsize=18)
plt.ylabel('Число людей\n',fontsize=18)

Больше половины населения привито в Гибралтаре, Сейшелах и Израиле, что также связано с их небольшими населениями. Особенно хочется выделить Великобританию, население которого  в 7 раз выше, чем в вышеуказанных странах и США, занимающую третье место по числу населения в мире. В Великобритании привито более 40 % населения, в США чуть более 20 %.

# Вывод

Наибольшее число вакцин принадлежит населению Гибралтара, Сейшел и Израиля, там же и привито более половины населения. Следовательно эти страны обладают наилучшими программами вакцинации. Среди высоконаселенных стран лучшими показателями обладают Великобритания и США.